# DST Airlines

[LINK](https://docs.google.com/document/d/1wyo5QOe8UZZE0_fm0PNdc_bTUNqjyrBMHSHwH-JfSEM/edit)

Description détaillée :  
De nos jours, il est possible d’avoir des informations sur les vols dans le monde entier et de traquer en temps réel un avion. Nous pouvons observer ce site en guise d’exemple.  
Le but ici est de s’en approcher le plus possible en passant par des API de différentes compagnies aériennes.

Etape | Description | Modules / Masterclass / Templates | Objectif | Conditions de validation du projet
----- | ----------- | --------------------------------- | -------- | ---------------------------------
1 | Exploration de données non structurées | Passer par l’API de Lufthansa pour récupérer des données sur les vols | 103 - Introduction aux APIs de données | Un fichier explicatif du traitement et des différentes données accessibles (doc / pdf). Un exemple de données collectées.


[Lufthansa Open API Documentation](https://developer.lufthansa.com/docs)

[LH OpenAPI testing](https://developer.lufthansa.com/io-docs)

[Public API Method Details](https://developer.lufthansa.com/docs/read/api_details)

!!! [Un repo GIT englobe déjà les requêtes à l'API, il suffirait de le forker](https://github.com/arossmann/lh_open_api)

In [2]:
from Lhapi import Lhapi as api

test = api()


In [ ]:
# to test a request
countries = test.request_api('https://api.lufthansa.com/v1/flight-schedules/flightschedules/passenger?airlines=LH&startDate=01JAN24&endDate=01JAN24&daysOfOperation=1234567&timeMode=UTC')
print(countries)

In [ ]:
# get all files for countries reference
test.request_full_files('https://api.lufthansa.com/v1/mds-references/countries?lang=EN&limit=100', 'countries')


In [ ]:
# get all files for cities reference
# 10m4s
test.request_full_files('https://api.lufthansa.com/v1/mds-references/cities?lang=EN&limit=100', 'cities')

In [ ]:
# get all files for airports reference
# 13m43s
test.request_full_files('https://api.lufthansa.com/v1/mds-references/airports?lang=EN&limit=100', 'airports')

In [ ]:
# get all files for airlines reference
# 7s
test.request_full_files('https://api.lufthansa.com/v1/mds-references/airlines?lang=EN&limit=100', 'airlines')

In [ ]:
# get all files for aircraft reference
# 2.7s
test.request_full_files('https://api.lufthansa.com/v1/mds-references/aircraft?lang=EN&limit=100', 'aircraft')

In [ ]:
# REFERENCES
# https://developer.lufthansa.com/docs/read/api_details/Reference_Data
# get all references

reference_items = ['countries', 'cities', 'airports', 'airlines', 'aircraft']
for ref in reference_items:
    test.request_full_files(f'https://api.lufthansa.com/v1/mds-references/{ref}?lang=EN&limit=100', ref)

In [ ]:
# get lounges from airports where LH is present
import json

def read_file(file_path, word):
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip().startswith('"' + word + '"'):
                res = line.split(':')[1].replace(',', '').replace('"', '').strip()
                yield res


airport_lh_codes = read_file("files/airports_lh.json", "AirportCode")
for code in airport_lh_codes:
    print(test.request_api("/offers/lounges/" + code))


In [ ]:
# Request schedules by line and by day
# https://api.lufthansa.com/v1/flight-schedules/flightschedules/passenger?airlines=LH&startDate=01JAN24&endDate=01JAN24&daysOfOperation=1234567&timeMode=UTC
# This API provides Lufthansa Groups flight schedule information (limited to LH - Lufthansa, EN - Air Dolomiti, LX - Swiss, OS - Austrian, WK - Edelweiss, SN - Brussels Airlines and  4Y - Eurowings Discover). 

dt_range = test.gen_dates_range('01DEC23', '30JAN24')
airline = 'LX'

for dt in dt_range:
    print(dt)
    for airline in ['OS', 'WK', 'SN', '4Y']:
        test.request_full_files(f"https://api.lufthansa.com/v1/flight-schedules/flightschedules/passenger?airlines={airline}&startDate={dt}&endDate={dt}&daysOfOperation=1234567&timeMode=UTC", f"flightschedules_{airline}_{dt}")


In [ ]:
# request flight status
# The available date range is from 7 days in the past until 5 days in the future. 
# GET /operations/flightstatus/{flightNumber}/{date}
# SELECT *
# FROM DB_SL_EP_LAB.DST.DST_FLIGHT_SCHEDULES
# WHERE startdate = '31JAN24'
# AND airlineid = 'LX'
# AND flightnumber = 8
# ;

In [ ]:
status = test.request_api('https://api.lufthansa.com/v1/operations/flightstatus/LX8/2024-01-31')
print(status)


In [ ]:
status_route = test.request_api('https://api.lufthansa.com/v1/operations/flightstatus/route/FRA/ZRH/2024-01-31')
print(status_route)

In [ ]:
status_arrivals = test.request_api('https://api.lufthansa.com/v1/operations/flightstatus/arrivals/ZRH/2024-01-31')
print(status_arrivals)

In [ ]:
customerflightinformation = test.request_api('https://api.lufthansa.com/v1/operations/customerflightinformation/LX8/2024-01-31')
print(customerflightinformation)

In [ ]:
customerflightinformation_arrivals = test.request_api('https://api.lufthansa.com/v1/operations/customerflightinformation/arrivals/FRA/2024-01-31T18:00')
print(customerflightinformation_arrivals)